In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import BertModel
from tokenizers import BertWordPieceTokenizer

import torch
import torch.nn.utils.prune as prune
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix

device = None

device = None

if torch.cuda.is_available():
    device = torch.device("cuda")
    print (device)
else:
    print ("cuda device not found.")


# 모델 및 토크나이저 로드
# model = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
# model = model.to(device)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")




class TestDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.sentences = df.iloc[:, 1:].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1).values
        self.labels = df.iloc[:, 0].values
        self.tokenizer = tokenizer
  
    def __len__(self):
        return len(self.sentences)
    
    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        inputs = self.tokenizer(sentence, truncation=True, max_length=512, padding='max_length', return_tensors="pt")
        label = torch.tensor(self.labels[idx]) # label은 1부터 시작하기 때문에, 나중에 inference할 때에 예측값에 1을 더해줘야 합니다.
        
        return inputs, label

test_data = "b.csv"
test_df = pd.read_csv(test_data, header=None)

test_dataset = TestDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

class_1_data = np.array([
    [-0.07, 0.30, 0.07, -0.10, 0.03, -0.30, 0.13, 0.03, -0.27, -0.53, -0.20, -0.13],
    [-0.20, -0.10, 0.00, 0.00, 0.03, 0.00, -0.20, -0.37, 0.03, 0.27, -0.47, -0.10],
    [0.03, 0.17, 0.07, -0.23, 0.00, 0.10, -0.40, -0.13, -0.17, 0.10, -0.47, -0.03],
    [-1.03, 0.13, 0.13, -0.23, -0.07, 0.07, 0.03, 0.03, -0.20, 0.13, -0.23, -0.27],
    [-1.03, -0.33, -0.20, -0.40, 0.17, -0.10, 0.10, 0.10, -0.13, 0.03, -0.13, -0.13],
    [-0.03, -0.13, 0.07, -0.03, 0.33, 0.00, 0.10, 0.03, -0.27, -0.23, -0.23, -0.17],
    [-0.27, 0.03, -0.10, 0.00, -0.13, -0.33, -0.10, 0.40, 1.03, -0.10, 0.13, 0.23],
    [-0.57, 0.07, 0.10, 0.03, 0.17, 0.70, 0.07, -0.07, -0.57, 0.07, -0.03, 0.10],
    [-0.13, -0.37, -0.03, -0.17, -0.30, 0.03, -0.33, 0.20, 0.13, -0.43, -0.40, -0.50],
    [-0.17, -0.17, 0.13, -0.27, -0.40, 1.77, -0.07, 0.03, 0.40, 0.13, 0.17, -0.30],
    [0.60, -0.30, -0.03, 0.07, -0.03, 0.00, 0.53, -0.03, 0.43, -0.03, 0.13, 0.80],
    [0.50, 0.17, -0.47, -0.10, -0.77, 0.10, 0.60, -0.73, -0.50, -0.03, -0.10, 1.10]
])
class_2_data = np.array([
    [-0.10, -0.07, -0.27, -0.27, 0.27, -0.07, -0.17, -0.33, -0.50, -0.03, -0.17, 0.07],
    [0.07, 0.03, -0.17, 0.10, -0.17, -0.23, -0.17, -0.23, -0.07, -0.33, -0.10, -0.27],
    [-0.17, -0.13, 0.03, 0.10, 0.03, 0.00, -0.13, -0.20, 0.03, -0.07, -0.10, 0.00],
    [0.20, 0.10, 0.07, -0.07, -0.20, -0.43, 0.17, -0.10, -0.37, -0.13, -0.27, -0.07],
    [-0.17, -0.07, 0.10, -0.27, -0.20, 0.00, -0.03, -0.20, 0.00, 0.23, 0.00, 0.10],
    [-0.10, 0.03, 0.07, 0.03, 0.10, -0.10, 0.17, -0.13, -0.23, 0.17, 0.17, -0.03],
    [-0.17, -0.03, 0.13, 0.00, -0.03, -0.13, -0.03, 0.00, -0.47, -0.23, -0.03, 0.30],
    [0.17, 0.03, 0.13, -0.07, -0.23, -0.13, 0.00, 0.00, 0.00, -0.13, 0.03, 0.07],
    [-0.03, -0.13, -0.03, 0.03, -0.03, -0.10, -0.13, -0.27, -0.07, 0.00, 0.07, -0.03],
    [-0.03, -0.70, 0.27, 0.23, 0.10, -1.30, -0.13, -0.10, -0.47, -0.17, 0.03, -0.23],
    [0.00, -0.23, 0.13, 0.17, -0.07, 0.03, -0.57, -0.43, -0.43, 0.00, -0.03, -0.93],
    [-1.53, 0.07, 0.17, 0.03, 0.03, -0.20, -0.60, -0.07, 0.37, 0.37, -0.23, -0.37]
])
class_3_data = np.array([
    [0.23, 0.33, 0.03, -0.10, 0.07, -0.67, 0.17, 0.30, 0.27, -0.43, 0.00, 0.00],
    [0.17, -0.03, 0.20, 0.07, 0.00, 0.00, 0.07, 0.07, 0.20, 0.10, -0.07, 0.07],
    [-0.10, -0.10, 0.20, 0.03, -0.03, -0.07, 0.13, -0.03, -0.03, -0.17, 0.00, 0.10],
    [-0.20, 0.00, -0.13, 0.00, 0.20, 0.10, -0.07, -0.13, -0.03, 0.07, -0.07, -0.20],
    [-0.03, 0.10, -0.20, 0.20, 0.10, -0.27, -0.13, -0.07, -0.07, -0.07, -0.03, 0.03],
    [-0.03, -0.13, -0.07, -0.10, 0.10, 0.03, -0.07, 0.00, 0.13, 0.00, -0.17, 0.00],
    [0.03, -0.17, -0.07, -0.03, -0.03, -0.07, -0.07, -0.10, 0.00, -0.07, 0.03, 0.03],
    [-0.23, -0.13, 0.00, 0.03, 0.07, -0.17, -0.07, 0.00, -0.03, -0.10, -0.20, 0.03],
    [-0.03, -0.10, 0.13, -0.07, 0.03, -0.13, -0.10, 0.07, -0.10, 0.00, -0.33, -0.07],
    [0.30, -0.03, -0.10, 0.13, 0.07, -0.20, -0.07, -0.40, -0.50, -0.03, -0.13, 0.13],
    [0.20, 0.10, -0.20, 0.10, -0.23, 0.07, -1.07, 0.33, 0.27, 0.07, -0.03, -0.20],
    [-0.67, 0.07, 0.00, 0.23, -0.23, 0.23, 0.33, -0.07, -0.17, 0.27, 0.07, -0.83]
])
class_4_data = np.array([
    [0.33, 0.20, -0.07, -0.13, -0.10, 0.10, -0.03, 0.23, -0.17, -0.17, -0.30, 0.00],
    [-0.07, -0.10, 0.33, 0.07, -0.10, -0.13, 0.03, 0.13, 0.03, 0.03, -0.27, 0.23],
    [0.07, -0.43, 0.13, -0.07, 0.10, 0.17, 0.03, -0.13, -0.17, -0.10, 0.07, 0.23],
    [0.50, -0.03, -0.03, 0.20, 0.00, 0.10, -0.17, -0.03, -0.17, -0.03, -0.20, 0.07],
    [-0.17, -0.17, 0.23, -0.07, -0.13, 0.07, -0.23, -0.03, 0.10, 0.13, -0.17, -0.13],
    [-0.07, -0.07, -0.17, 0.00, -0.17, -0.13, -0.07, 0.03, -0.17, 0.10, -0.03, 0.10],
    [0.13, -0.03, -0.17, 0.00, -0.07, -0.07, 0.00, -0.27, 0.30, 0.10, -0.03, 0.07],
    [0.10, -0.10, -0.10, -0.10, 0.03, -0.50, -0.03, 0.00, 0.10, -0.20, -0.23, 0.03],
    [0.07, 0.33, -0.17, 0.10, -0.03, 0.13, 0.03, -0.07, -0.43, -0.13, 0.10, -0.10],
    [0.20, 0.33, -0.17, 0.10, -0.23, -0.87, 0.13, -0.60, -0.07, 0.10, 0.50, -0.10],
    [-0.70, 0.20, -0.03, 0.67, 0.27, 0.03, 1.47, 0.97, -0.07, -0.03, -0.07, 0.27],
    [1.80, 0.30, -0.13, -0.13, 0.57, -0.20, 0.47, 0.03, 0.37, -0.40, 0.03, 0.93]
])
class_5_data = np.array([
    [-0.20, -0.37, -0.10, 0.20, -0.07, 0.33, 0.17, -0.10, 0.03, 0.20, -0.03, 0.03],
    [0.03, 0.03, 0.30, -0.10, -0.23, -0.30, 0.00, -0.27, -0.17, -0.10, 0.17, -0.13],
    [-0.33, 0.03, 0.03, -0.13, -0.10, -0.33, -0.10, -0.17, -0.17, -0.27, 0.07, 0.07],
    [-0.43, -0.10, -0.10, -0.17, -0.13, 0.03, 0.00, -0.10, 0.10, -0.03, 0.13, -0.23],
    [-0.33, 0.13, 0.13, -0.07, 0.10, 0.07, -0.07, -0.07, -0.07, 0.20, -0.03, 0.10],
    [0.10, 0.00, -0.17, -0.13, 0.03, -0.10, -0.03, 0.00, 0.07, 0.03, -0.10, -0.03],
    [0.07, 0.07, -0.17, 0.00, -0.13, 0.17, 0.13, -0.07, 0.20, 0.03, -0.10, 0.00],
    [-0.03, -0.23, 0.13, -0.03, -0.13, -0.37, -0.10, 0.13, -0.07, -0.10, 0.20, -0.20],
    [-0.03, 0.07, 0.00, 0.03, 0.03, -0.03, -0.20, 0.00, 0.10, 0.03, 0.23, 0.27],
    [0.07, 0.57, 0.00, -0.20, -0.13, -1.30, -0.13, 0.20, 0.30, 0.00, -0.53, -0.33],
    [0.63, 0.13, -0.30, -0.63, -0.53, 0.03, -0.67, -0.90, 0.20, 0.17, 0.00, -0.07],
    [-0.57, -0.23, -0.20, 0.30, 0.03, 0.03, -0.73, -0.70, -0.37, 0.60, -0.53, 1.37]
])
class_6_data = np.array([
    [0.10, -0.17, 0.03, 0.00, 0.10, 0.27, -0.10, -0.07, 0.27, 0.23, 0.07, -0.03],
    [0.00, 0.00, 0.03, -0.03, 0.00, -0.03, -0.03, -0.07, 0.13, 0.03, -0.10, -0.23],
    [0.10, 0.07, 0.07, 0.03, 0.07, -0.03, -0.10, 0.07, 0.00, 0.03, 0.00, -0.17],
    [-0.07, 0.00, -0.03, -0.07, 0.03, 0.07, 0.00, -0.10, -0.13, -0.07, -0.07, 0.00],
    [0.03, 0.03, 0.13, 0.03, 0.07, 0.03, -0.07, -0.07, 0.00, 0.00, -0.10, 0.03],
    [-0.03, 0.00, 0.00, 0.00, 0.00, -0.07, -0.03, -0.07, -0.07, -0.10, 0.03, -0.03],
    [0.00, -0.10, 0.00, 0.00, 0.00, -0.03, -0.03, -0.03, -0.17, 0.00, 0.00, 0.03],
    [0.07, -0.07, -0.07, -0.03, -0.10, -0.20, -0.03, 0.03, -0.03, 0.03, 0.03, 0.00],
    [-0.07, 0.03, -0.03, -0.03, -0.07, -0.03, -0.03, -0.03, -0.03, -0.03, -0.03, 0.07],
    [-0.10, -0.27, 0.03, 0.03, 0.00, -0.60, -0.07, 0.13, -0.10, 0.00, -0.03, 0.13],
    [0.00, -0.07, 0.13, 0.03, 0.13, -0.03, -1.00, -0.13, -0.10, 0.00, 0.00, 0.03],
    [-0.10, -0.13, 0.03, -0.07, -0.03, -0.03, 0.13, 0.03, 0.07, 0.03, 0.07, -1.00]
])
class_7_data = np.array([
    [0.20, -0.23, -0.17, -0.27, 0.00, -0.33, 0.07, -0.30, -0.30, -0.17, 0.13, -0.50],
    [-0.13, -0.13, -0.30, 0.17, 0.17, -0.03, -0.07, 0.13, 0.00, -0.10, -0.13, 0.23],
    [-0.30, 0.03, -0.13, 0.13, -0.07, -0.13, -0.10, 0.23, -0.07, -0.20, -0.30, 0.07],
    [-0.13, 0.30, -0.20, 0.03, 0.23, -0.27, -0.17, 0.07, -0.53, -0.07, -0.17, -0.03],
    [-0.43, -0.07, -0.03, -0.13, -0.20, -0.03, -0.03, -0.10, 0.07, 0.27, -0.23, 0.07],
    [0.03, 0.03, -0.10, -0.10, -0.17, -0.10, -0.03, -0.30, 0.10, 0.00, -0.23, -0.20],
    [-0.07, -0.23, -0.10, -0.10, 0.07, -0.27, -0.10, 0.03, -0.30, 0.10, 0.13, 0.07],
    [-0.47, -0.07, -0.07, -0.13, -0.13, -0.13, -0.13, -0.20, 0.13, -0.07, 0.03, -0.13],
    [-0.07, -0.03, -0.03, -0.03, -0.07, 0.00, -0.07, 0.23, -0.17, -0.57, -0.20, -0.67],
    [0.00, 0.07, -0.20, -0.03, 0.03, 0.60, 0.17, -0.17, -0.77, -0.03, -0.07, 0.53],
    [0.03, -0.37, 0.03, -0.30, 0.60, -0.30, 0.53, 0.77, -0.03, -0.20, 0.03, -0.10],
    [0.17, 0.13, 0.13, 0.50, -0.27, -0.27, 0.97, 0.90, 0.03, -0.13, 0.40, -2.33]
])
class_8_data = np.array([
    [0.00, -0.20, -0.23, -0.03, -0.17, 0.27, -0.37, -0.03, -0.17, 0.03, -0.40, -0.03],
    [-0.53, -0.07, 0.03, -0.07, -0.23, -0.23, -0.20, 0.03, -0.10, 0.10, -0.10, 0.33],
    [-0.17, -0.07, -0.10, -0.07, -0.03, -0.07, 0.33, 0.13, 0.20, -0.07, 0.17, -0.13],
    [0.33, -0.10, 0.03, 0.03, -0.17, -0.33, 0.03, 0.00, 0.30, -0.23, -0.17, -0.17],
    [0.27, 0.00, 0.43, -0.30, -0.33, 0.07, -0.13, -0.23, 0.00, -0.07, -0.03, 0.03],
    [-0.07, -0.23, -0.20, 0.13, -0.20, -0.13, 0.03, -0.10, 0.07, 0.00, -0.23, 0.03],
    [-0.20, -0.03, 0.20, 0.00, 0.03, 0.07, -0.13, -0.13, 0.13, -0.17, 0.00, 0.03],
    [-0.10, -0.20, -0.17, 0.13, -0.13, 0.20, -0.07, -0.13, 0.17, 0.03, 0.20, -0.07],
    [-0.03, 0.10, 0.10, 0.00, -0.03, -0.03, 0.13, 0.07, -0.17, 0.10, 0.23, 0.10],
    [0.03, 0.13, 0.00, 0.03, 0.20, 0.60, -0.10, 0.37, 0.57, 0.13, -0.40, 0.03],
    [-0.23, 0.27, -0.30, -0.27, 0.93, 0.10, -0.60, 0.07, 0.00, 0.03, -0.13, 0.00],
    [-0.50, -0.07, -0.13, 0.00, 0.40, 0.20, -0.60, 0.27, -0.13, -0.30, 0.00, 1.37]
])
class_9_data = np.array([
    [-0.40, -0.27, -0.13, -0.07, 0.00, -0.73, -0.37, -0.27, 0.20, -0.53, -0.10, -0.20],
    [0.07, 0.00, 0.07, -0.30, -0.10, -0.13, -0.03, 0.17, -0.30, -0.03, -0.27, -0.07],
    [-0.13, 0.13, -0.10, 0.00, -0.03, -0.23, 0.00, -0.33, -0.10, -0.33, -0.07, -0.07],
    [0.00, -0.17, -0.10, 0.03, -0.03, -0.07, 0.07, -0.20, 0.13, -0.13, -0.20, -0.07],
    [0.37, 0.07, -0.13, 0.10, -0.20, -0.17, 0.07, 0.00, -0.10, -0.20, -0.03, -0.10],
    [-0.13, -0.07, -0.13, -0.03, -0.17, -0.03, -0.17, 0.00, -0.17, -0.07, -0.13, 0.00],
    [0.20, -0.27, -0.27, -0.07, -0.07, -0.17, -0.10, -0.27, -0.43, 0.07, -0.10, -0.07],
    [0.13, -0.07, -0.17, -0.07, -0.20, -0.27, 0.00, 0.03, -0.03, -0.20, -0.13, -0.27],
    [-0.07, 0.07, -0.13, -0.20, 0.03, -0.17, -0.20, -0.27, -0.27, 0.23, 0.37, -0.07],
    [-0.10, -0.03, -0.27, -0.13, -0.07, -0.77, 0.03, 0.03, 0.00, -0.27, -0.13, -0.20],
    [-0.10, -0.30, -0.23, -0.97, -0.93, 0.00, 0.50, -0.33, -0.03, -0.07, -0.10, 0.00],
    [0.40, -0.20, 0.33, -0.43, -0.13, -0.83, -0.93, 0.40, -0.20, 0.07, 0.00, -1.13]
])
class_10_data = np.array([
    [-0.07, 0.07, 0.07, 0.13, 0.00, 0.27, -0.07, 0.10, 0.23, 0.63, -0.07, 0.00],
    [-0.07, -0.07, -0.37, -0.03, -0.13, -0.03, -0.03, -0.13, 0.00, -0.17, -0.07, 0.00],
    [-0.13, 0.20, -0.10, -0.10, 0.10, 0.20, -0.10, 0.07, 0.00, -0.03, 0.20, -0.20],
    [0.03, 0.07, -0.07, 0.10, 0.00, 0.07, -0.10, 0.03, 0.17, 0.00, -0.03, 0.17],
    [-0.10, -0.07, -0.20, 0.13, 0.03, -0.10, 0.03, -0.03, -0.13, -0.13, 0.00, 0.00],
    [0.10, -0.03, -0.07, -0.03, -0.07, 0.00, 0.00, 0.03, -0.03, 0.07, 0.13, -0.13],
    [0.33, -0.10, 0.07, 0.07, -0.10, 0.17, 0.07, -0.03, -0.07, -0.03, 0.03, 0.07],
    [0.63, 0.07, 0.00, -0.07, 0.20, -0.27, -0.07, -0.03, 0.07, 0.17, -0.10, -0.03],
    [-0.07, 0.13, 0.10, 0.00, -0.03, 0.00, 0.03, -0.03, 0.40, 0.33, 0.33, 0.20],
    [-0.20, -0.33, 0.30, 0.07, 0.23, -0.60, 0.00, 0.13, 0.37, -0.07, -0.03, 0.03],
    [-0.07, -0.03, 0.37, 0.20, -0.37, -0.07, 0.90, -1.03, -0.03, 0.00, 0.07, 0.03],
    [-0.43, -0.17, -0.03, -0.07, -0.17, 0.23, -0.13, -0.37, 0.30, -0.07, 0.33, 0.33]
])

class_1_neg_acc = np.array(
[
    [-0.01, -0.09, -0.03, -0.06, -0.04, -0.02, -0.09, -0.06, 0.00, 0.00, -0.03, -0.03],
    [-0.06, -0.05, -0.03, -0.06, -0.05, -0.09, -0.03, -0.04, -0.03, -0.02, -0.01, 0.05],
    [-0.08, -0.06, -0.01, -0.02, 0.00, -0.06, 0.04, -0.04, -0.01, -0.06, 0.03, -0.06],
    [0.13, -0.03, -0.07, -0.01, 0.02, -0.03, 0.01, -0.02, -0.02, -0.02, -0.03, -0.03],
    [0.07, 0.06, 0.07, -0.04, -0.11, -0.03, -0.04, -0.02, -0.02, 0.00, 0.00, 0.00],
    [-0.01, -0.02, -0.05, -0.02, -0.05, -0.08, -0.07, 0.01, 0.00, 0.01, -0.04, 0.01],
    [-0.01, -0.05, -0.06, 0.00, 0.00, 0.01, -0.03, -0.12, -0.15, -0.03, -0.03, 0.04],
    [0.06, -0.04, -0.05, -0.03, -0.07, -0.22, -0.01, -0.01, 0.13, -0.04, 0.00, -0.02],
    [-0.02, 0.09, -0.05, -0.02, 0.03, -0.02, 0.05, -0.05, -0.12, 0.01, 0.10, 0.01],
    [0.04, -0.06, -0.01, 0.05, 0.01, -0.64, -0.05, -0.03, 0.00, -0.05, -0.09, 0.00],
    [-0.09, 0.05, -0.06, -0.06, 0.03, -0.01, -0.22, -0.02, -0.07, -0.03, -0.03, -0.22],
    [-0.20, 0.02, 0.05, 0.02, 0.09, -0.02, -0.25, 0.05, 0.05, -0.03, 0.05, -0.30]
]   
)
class_2_neg_acc = np.array(
[
    [0.05, -0.02, -0.01, 0.05, -0.03, 0.02, -0.01, 0.03, 0.10, -0.04, 0.01, -0.04],
    [-0.01, 0.02, 0.01, -0.02, 0.01, 0.00, 0.00, 0.08, 0.00, 0.04, -0.01, 0.05],
    [0.00, -0.01, 0.01, 0.01, 0.00, -0.02, 0.01, 0.03, 0.00, -0.01, 0.02, 0.02],
    [-0.04, -0.01, -0.02, 0.01, -0.01, 0.03, -0.02, 0.00, 0.04, -0.01, -0.01, 0.00],
    [0.03, 0.00, -0.01, 0.03, 0.00, 0.03, 0.04, 0.02, 0.00, 0.00, -0.02, 0.01],
    [0.00, -0.01, -0.02, -0.02, -0.02, -0.01, 0.00, 0.01, -0.02, -0.02, -0.01, 0.00],
    [0.03, -0.01, 0.00, 0.00, -0.01, 0.01, 0.01, 0.00, 0.05, 0.00, 0.01, -0.01],
    [-0.01, -0.01, -0.02, 0.02, -0.01, 0.00, 0.02, 0.00, -0.01, -0.01, 0.02, -0.01],
    [0.01, 0.01, 0.02, 0.00, 0.02, 0.01, 0.01, 0.02, -0.02, 0.01, -0.01, 0.01],
    [0.00, 0.04, -0.01, 0.01, -0.02, 0.12, 0.03, 0.00, 0.08, 0.03, 0.00, 0.04],
    [0.01, 0.04, -0.02, -0.04, -0.01, 0.02, 0.06, 0.06, 0.04, 0.00, 0.00, 0.13],
    [0.21, 0.03, -0.05, -0.01, 0.02, 0.02, 0.05, 0.02, -0.05, -0.04, 0.02, 0.05]
]
)
class_3_neg_acc = np.array(
[
    [-0.05, -0.04, -0.02, 0.00, 0.04, 0.12, -0.01, -0.04, -0.02, 0.08, -0.02, -0.02],
    [-0.01, 0.00, -0.02, 0.00, -0.01, 0.02, -0.02, 0.02, 0.00, -0.01, -0.01, 0.02],
    [0.02, 0.03, -0.02, 0.02, 0.01, 0.02, -0.01, 0.02, 0.01, 0.02, 0.02, 0.00],
    [0.02, 0.03, 0.05, 0.02, -0.05, -0.04, 0.01, 0.01, 0.01, 0.01, -0.01, 0.02],
    [0.03, -0.02, 0.04, -0.04, -0.01, 0.02, 0.03, 0.03, 0.00, 0.00, 0.02, 0.03],
    [0.00, 0.00, 0.00, 0.04, -0.03, 0.02, 0.02, -0.01, 0.01, 0.06, 0.02, -0.02],
    [0.04, 0.06, 0.04, 0.01, 0.01, 0.07, -0.03, 0.02, 0.02, 0.05, 0.04, 0.05],
    [0.02, 0.02, 0.02, 0.00, -0.01, 0.07, 0.00, -0.01, 0.02, -0.02, 0.06, -0.01],
    [0.00, 0.03, 0.02, 0.02, 0.02, 0.01, -0.02, -0.01, 0.01, 0.06, 0.12, 0.00],
    [-0.05, 0.03, 0.03, -0.05, 0.00, 0.11, 0.03, 0.05, 0.10, 0.04, 0.01, -0.03],
    [-0.04, -0.01, 0.07, -0.03, 0.07, 0.00, 0.28, -0.10, 0.00, 0.00, 0.01, 0.06],
    [0.19, -0.01, -0.01, -0.04, 0.07, -0.07, -0.10, 0.04, 0.05, -0.04, 0.00, 0.30]
]
)
class_4_neg_acc = np.array(
[
    [-0.04, -0.03, 0.01, 0.02, 0.03, -0.09, -0.05, 0.00, 0.02, -0.03, -0.01, 0.02],
    [0.03, 0.02, 0.02, -0.01, -0.01, 0.00, 0.00, -0.04, -0.02, 0.01, -0.01, -0.06],
    [-0.04, 0.00, 0.03, 0.00, 0.00, -0.01, -0.01, 0.01, 0.00, -0.04, -0.04, 0.01],
    [-0.09, 0.02, -0.03, -0.03, -0.02, -0.01, 0.02, 0.00, 0.01, -0.03, -0.04, -0.02],
    [-0.09, -0.01, -0.01, -0.02, -0.01, -0.03, 0.02, -0.05, -0.02, -0.02, -0.02, -0.03],
    [0.01, -0.01, 0.00, -0.01, 0.02, 0.00, 0.02, -0.01, 0.02, 0.01, -0.04, -0.02],
    [-0.03, -0.04, -0.02, -0.01, 0.00, -0.03, 0.00, 0.05, -0.01, -0.04, 0.00, 0.01],
    [0.00, 0.02, 0.02, 0.00, 0.00, 0.05, -0.02, -0.01, -0.03, 0.00, 0.00, -0.03],
    [-0.01, -0.03, 0.03, -0.04, -0.03, -0.01, -0.03, 0.03, 0.08, 0.02, 0.00, 0.03],
    [-0.03, -0.06, 0.03, -0.03, 0.04, 0.14, -0.02, 0.06, -0.05, -0.04, -0.06, 0.05],
    [0.18, -0.08, 0.02, -0.16, -0.06, 0.00, -0.21, -0.20, 0.03, 0.01, 0.01, -0.02],
    [-0.31, -0.06, 0.03, 0.05, -0.09, -0.02, -0.16, -0.01, -0.07, 0.08, 0.02, -0.19]
]  
)
class_5_neg_acc = np.array(
[
    [0.03, 0.04, 0.03, -0.02, -0.01, -0.04, 0.00, 0.00, -0.03, 0.01, -0.01, -0.01],
    [-0.01, 0.00, -0.02, 0.03, 0.00, 0.00, 0.00, 0.02, 0.01, 0.01, -0.02, 0.02],
    [0.02, 0.01, -0.01, -0.01, 0.00, 0.02, -0.01, 0.00, 0.01, 0.02, 0.00, 0.00],
    [0.02, 0.01, 0.01, 0.02, -0.01, 0.00, -0.02, 0.00, -0.01, 0.02, -0.02, 0.02],
    [0.03, -0.01, 0.01, 0.00, 0.02, 0.02, -0.01, 0.01, 0.01, 0.02, 0.02, 0.00],
    [0.00, 0.01, 0.00, 0.01, -0.01, -0.01, 0.02, 0.02, -0.01, 0.00, 0.01, 0.00],
    [0.00, -0.01, 0.01, 0.00, 0.00, -0.02, 0.00, 0.01, 0.01, -0.01, 0.02, 0.01],
    [-0.01, 0.00, 0.00, 0.00, 0.02, 0.03, 0.01, 0.01, -0.03, 0.00, -0.02, 0.02],
    [0.00, 0.00, -0.01, 0.00, 0.00, 0.01, 0.01, 0.00, -0.02, -0.01, 0.00, -0.05],
    [0.03, -0.06, -0.01, -0.01, 0.02, 0.15, 0.00, -0.01, -0.04, 0.00, 0.05, 0.03],
    [-0.07, 0.01, 0.01, 0.05, 0.03, 0.00, 0.13, 0.09, 0.00, 0.00, -0.01, 0.01],
    [0.03, 0.01, 0.01, -0.03, 0.00, 0.00, 0.07, 0.07, 0.02, -0.03, 0.04, -0.13]
] 
)
class_6_neg_acc = np.array(
[
    [-0.01, 0.02, -0.02, 0.00, 0.01, -0.04, 0.02, 0.02, -0.02, -0.02, -0.02, 0.01],
    [0.00, 0.01, 0.00, 0.00, 0.00, 0.01, 0.01, 0.00, -0.02, 0.00, -0.01, 0.00],
    [0.01, 0.01, -0.01, 0.00, 0.00, 0.01, 0.00, -0.02, 0.00, 0.00, 0.01, 0.01],
    [0.00, 0.01, 0.01, 0.00, 0.01, -0.01, 0.01, 0.01, 0.02, -0.01, 0.02, -0.01],
    [-0.03, 0.00, -0.02, 0.03, 0.01, 0.01, 0.01, 0.00, 0.02, 0.01, 0.00, 0.02],
    [0.01, 0.01, 0.00, 0.01, 0.00, 0.01, 0.01, 0.02, -0.02, 0.01, 0.00, 0.01],
    [0.02, 0.01, 0.01, 0.01, 0.01, -0.01, 0.02, 0.00, 0.06, 0.01, 0.01, 0.01],
    [0.01, 0.01, 0.00, 0.02, 0.02, 0.02, 0.01, 0.00, -0.01, 0.01, -0.01, 0.01],
    [0.01, 0.00, 0.01, 0.01, 0.00, 0.00, 0.00, 0.01, 0.00, 0.01, 0.03, -0.01],
    [0.02, 0.03, 0.00, 0.01, -0.01, 0.08, 0.01, 0.00, 0.06, 0.01, 0.01, -0.02],
    [0.02, 0.00, 0.00, 0.01, -0.04, 0.01, 0.16, 0.01, 0.03, 0.01, 0.00, 0.03],
    [0.03, 0.01, 0.03, 0.00, -0.02, -0.01, 0.00, 0.02, 0.00, 0.02, 0.00, 0.14]
]  
)
class_7_neg_acc = np.array(
[
    [-0.02, 0.03, 0.01, 0.01, 0.00, 0.04, 0.00, 0.04, 0.02, 0.06, -0.02, 0.01],
    [-0.01, -0.03, 0.03, 0.01, -0.02, -0.01, -0.01, 0.01, 0.00, 0.00, -0.02, -0.03],
    [0.01, 0.03, 0.02, -0.01, -0.01, -0.02, 0.01, 0.00, 0.00, 0.00, 0.04, 0.00],
    [0.03, -0.02, 0.01, -0.02, -0.01, -0.02, 0.00, 0.00, 0.01, 0.01, 0.02, -0.02],
    [0.03, 0.00, 0.01, 0.00, -0.01, -0.01, -0.01, -0.01, -0.01, 0.00, 0.00, -0.03],
    [-0.04, -0.02, 0.00, 0.00, 0.00, -0.03, 0.00, -0.02, -0.02, -0.02, -0.01, -0.01],
    [0.02, 0.01, 0.00, 0.00, -0.03, 0.04, 0.01, -0.03, 0.00, 0.01, -0.01, 0.00],
    [0.05, -0.02, 0.00, 0.00, 0.03, 0.00, -0.01, 0.01, -0.04, -0.01, -0.01, 0.00],
    [-0.01, -0.02, -0.02, 0.00, -0.01, -0.01, -0.01, -0.07, 0.00, 0.03, 0.02, 0.04],
    [-0.01, 0.00, 0.02, 0.00, -0.01, -0.11, -0.03, 0.07, 0.10, -0.01, -0.02, -0.09],
    [0.00, 0.04, -0.02, -0.03, -0.05, 0.00, -0.03, -0.14, 0.02, 0.03, -0.01, -0.02],
    [-0.08, -0.03, -0.01, -0.06, 0.01, 0.03, -0.15, -0.11, 0.00, 0.00, -0.06, 0.17]
]
)
class_8_neg_acc = np.array(
[
    [0.02, -0.02, -0.02, -0.05, 0.01, -0.10, -0.02, -0.03, -0.01, -0.10, -0.01, 0.00],
    [0.02, 0.00, -0.04, -0.02, 0.00, -0.02, 0.00, -0.08, 0.00, -0.06, -0.01, -0.04],
    [-0.04, 0.00, 0.00, 0.01, 0.00, 0.03, -0.09, -0.06, -0.06, -0.06, -0.13, 0.00],
    [-0.13, -0.03, -0.02, -0.02, 0.02, 0.02, -0.05, -0.05, -0.09, -0.02, -0.09, 0.02],
    [-0.17, 0.00, -0.10, 0.04, 0.04, -0.03, -0.01, -0.03, 0.00, -0.01, -0.01, 0.00],
    [0.01, 0.01, -0.02, 0.00, 0.01, 0.01, 0.00, -0.04, -0.03, -0.03, 0.03, -0.03],
    [-0.02, -0.05, -0.03, -0.02, -0.05, -0.10, -0.01, -0.03, -0.03, -0.01, -0.03, -0.02],
    [-0.01, -0.03, -0.02, -0.03, -0.02, -0.20, -0.04, 0.01, -0.04, 0.01, -0.05, -0.02],
    [-0.01, -0.06, -0.03, 0.00, -0.05, -0.02, -0.07, 0.01, -0.02, -0.06, -0.05, -0.06],
    [-0.02, -0.09, -0.03, -0.03, -0.05, -0.35, 0.03, -0.14, -0.15, -0.08, 0.04, -0.07],
    [0.03, -0.12, 0.00, 0.04, -0.36, -0.02, 0.10, -0.05, -0.03, -0.02, 0.00, 0.00],
    [0.03, -0.02, 0.04, 0.02, -0.15, -0.07, 0.14, -0.08, 0.01, 0.12, 0.01, -0.33]
]
)
class_9_neg_acc = np.array(
[
    [0.05, 0.03, 0.00, -0.04, 0.01, 0.10, 0.10, 0.02, -0.08, 0.06, -0.02, -0.01],
    [-0.03, -0.02, 0.00, 0.04, -0.04, 0.00, -0.02, -0.02, 0.03, 0.01, -0.02, -0.02],
    [0.00, -0.01, -0.02, -0.03, 0.02, 0.01, -0.02, 0.01, 0.00, 0.02, -0.01, -0.02],
    [-0.03, 0.02, 0.00, 0.03, 0.00, 0.01, -0.01, 0.00, -0.03, 0.00, 0.01, -0.04],
    [-0.06, -0.04, 0.01, -0.08, 0.02, -0.03, -0.06, -0.03, -0.01, 0.02, -0.05, -0.01],
    [-0.02, -0.03, 0.01, -0.03, 0.02, -0.02, 0.00, -0.01, 0.02, -0.05, -0.03, 0.00],
    [-0.04, 0.00, 0.00, -0.01, 0.00, 0.01, 0.01, 0.04, 0.05, -0.01, 0.00, 0.01],
    [-0.07, -0.01, 0.01, -0.01, -0.02, 0.03, -0.02, -0.02, -0.01, 0.00, -0.02, 0.01],
    [-0.01, -0.02, 0.00, 0.01, -0.03, 0.00, -0.02, 0.03, 0.06, -0.06, -0.16, 0.00],
    [0.01, 0.00, 0.02, 0.04, -0.02, 0.11, -0.04, -0.01, -0.07, 0.04, 0.00, 0.03],
    [-0.02, 0.02, 0.01, 0.15, 0.17, -0.01, -0.12, 0.06, 0.00, -0.01, 0.04, -0.02],
    [-0.11, 0.01, -0.12, 0.07, 0.00, 0.12, 0.25, -0.10, 0.00, -0.04, -0.05, 0.31]
]
)
class_10_neg_acc = np.array(
[
    [0.01, 0.05, -0.01, 0.03, 0.01, -0.06, 0.01, -0.02, 0.01, -0.09, 0.03, 0.00],
    [0.03, 0.01, 0.08, 0.03, 0.04, -0.02, 0.02, 0.01, 0.03, 0.02, -0.02, 0.03],
    [0.00, 0.00, 0.03, 0.01, 0.02, -0.02, 0.04, 0.01, 0.02, 0.00, 0.02, 0.05],
    [0.02, 0.04, 0.03, 0.00, 0.05, 0.00, 0.05, 0.00, 0.01, 0.01, 0.03, -0.01],
    [-0.02, 0.00, 0.04, 0.00, 0.01, 0.01, 0.00, 0.01, 0.01, 0.05, 0.00, 0.03],
    [0.03, 0.01, 0.02, 0.01, 0.04, 0.04, 0.01, 0.00, 0.01, 0.03, 0.00, 0.02],
    [0.00, 0.01, 0.00, 0.02, 0.04, -0.03, 0.00, 0.04, 0.05, 0.03, 0.01, 0.01],
    [-0.06, 0.00, 0.02, 0.01, 0.00, 0.11, 0.02, 0.01, 0.00, 0.01, 0.02, 0.00],
    [0.01, 0.04, 0.02, 0.01, 0.02, 0.01, 0.00, 0.03, -0.01, -0.06, 0.00, -0.04],
    [0.04, 0.14, -0.03, 0.01, 0.02, 0.10, 0.02, -0.02, -0.02, 0.04, 0.00, 0.05],
    [0.04, -0.02, -0.03, -0.01, 0.21, 0.00, -0.13, 0.23, 0.02, 0.01, 0.00, 0.03],
    [0.12, 0.03, 0.01, 0.02, 0.02, -0.03, 0.10, 0.08, -0.03, 0.03, -0.01, -0.07]
]
)

def get_sorted_indices(data):

    data_np = np.array(data)
    data_flattened = data_np.flatten()
    sorted_indices = np.argsort(data_flattened)
    row_indices = sorted_indices // 12
    col_indices = sorted_indices % 12


    result = []


    for i in range(len(row_indices)):
        result.append((row_indices[i], col_indices[i]))

    return result
def get_sorted_indices_except_max(data):
    data_np = np.array(data)
    max_indices = np.argmin(data_np, axis=1)
    data_flattened = data_np.flatten()
    sorted_indices = np.argsort(data_flattened)[::-1]
    row_indices = sorted_indices // 12
    col_indices = sorted_indices % 12

    result = []

    for i in range(len(row_indices)):
        # 각 행의 최대값 인덱스를 제외
        if col_indices[i] != max_indices[row_indices[i]]:
            result.append((row_indices[i], col_indices[i]))

    return result


ablating_head_num_in_CI = 100


correct_num = []
correct_num_neg = []
for r in range(20, 20 + 1): # 20개 되살림
    print('recovered head num: ', r)

    # ================================================================================================
    # 1번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_1_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_1_neg_acc) ##############################

    ##########################
    model_1 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_1 = model_1.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_1.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    # 2번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_2_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_2_neg_acc) ##############################

    ##########################
    model_2 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_2 = model_2.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_2.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    # 3번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_3_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_3_neg_acc) ##############################

    ##########################
    model_3 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_3 = model_3.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_3.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    # 4번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_4_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_4_neg_acc) ##############################

    ##########################
    model_4 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_4 = model_4.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_4.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    # 5번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_5_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_5_neg_acc) ##############################

    ##########################
    model_5 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_5 = model_5.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_5.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #6번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_6_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_6_neg_acc) ##############################

    ##########################
    model_6 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_6 = model_6.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_6.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #7번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_7_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_7_neg_acc) ##############################

    ##########################
    model_7 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_7 = model_7.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_7.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #8번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_8_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_8_neg_acc) ##############################

    ##########################
    model_8 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_8 = model_8.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_8.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #9번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_9_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_9_neg_acc) ##############################

    ##########################
    model_9 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_9 = model_9.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_9.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================

    # ================================================================================================
    #10번 클래스 모듈에 대해서 프루닝
    prune_head_index = get_sorted_indices_except_max(class_10_data) ##############################
    prune_head_index = prune_head_index[:ablating_head_num_in_CI] # 100개의 head를 pruning
    recovering_head_index = get_sorted_indices(class_10_neg_acc) ##############################

    ##########################
    model_10 = AutoModelForSequenceClassification.from_pretrained("fabriceyhc/bert-base-uncased-yahoo_answers_topics")
    model_10 = model_10.to(device)
    ##########################

    recovering_head_num_in_TI = r
    actually_recovered_head_num = 0

    for i in recovering_head_index: # 제외할 헤드에서, 나중에 어차피 회복시킬 헤드를 제외시킴
        recovering_head_num_in_TI -= 1
        if i in prune_head_index:
            prune_head_index.remove(i)
            actually_recovered_head_num += 1      

        if recovering_head_num_in_TI == 0:
            break

    for layer_index, head_index in prune_head_index: # 헤드를 제외하는 부분
        model_10.bert.encoder.layer[layer_index].attention.prune_heads([head_index])
        ##########################

    print('actually_recovered_head_num: ', actually_recovered_head_num)
    # ================================================================================================


    preds = []
    true_labels = []

    for batch in tqdm(test_loader, desc="Evaluating"):
        inputs, labels = batch
        inputs = {k: v.squeeze(1).to(device) for k, v in inputs.items()} 
        labels = labels.to(device)

        with torch.no_grad():
            outputs_1 = model_1(**inputs)
            outputs_2 = model_2(**inputs)
            outputs_3 = model_3(**inputs)
            outputs_4 = model_4(**inputs)
            outputs_5 = model_5(**inputs)

            outputs_6 = model_6(**inputs)
            outputs_7 = model_7(**inputs)
            outputs_8 = model_8(**inputs)
            outputs_9 = model_9(**inputs)
            outputs_10 = model_10(**inputs)
            
        prediction = outputs_1.logits.argmax(dim=-1) + 1
        
        preds.extend(prediction.tolist())
        true_labels.extend(labels.tolist())
        
    # print('classification report')
    # print(classification_report(true_labels, preds, digits=5))

    # print('confusion matrix')
    # print(confusion_matrix(true_labels, preds))

    con = confusion_matrix(true_labels, preds)
    con_diag = np.diag(con) # numpy array
    con_vertical = np.sum(con, axis=0) # numpy array

    # print(con[0]) # TP ##############################
    # print(con_vertical[0]) # TP+ FP ##############################

    correct_num.append(con_diag[0].item()) ##############################
    correct_num_neg.append(con_vertical[0].item() - con_diag[0].item()) ##############################
    # for i in range(10):
    #     print(con[i])


    print(correct_num)
    print(correct_num_neg)

print('class 1') ##############################
print(correct_num)
print(correct_num_neg)